In [ ]:
import random
import numpy as np

In [ ]:
class Network(object):
    def __init__(self, sizes):
        """
        creates the network
        :param sizes: list containing the number of neurons in each layer, eg: [2, 3, 1] -> 2 neurons, 3 neurons, 1 neuron
        """
        self.num_layers = len(sizes)    # gives the number of layers of neurons
        self.sizes = sizes              # stores the sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # stores the random biases: [np.array([[-1.2323], [2.3434], ...]),
        #                           np.array([-1.2323], [2.3434], ...)] depending upon sizes, total sizes - 1 elements, each with y elements(for y in sizes[1:])
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        # stores the random weights, initialised such that there is a weight for each of the connections
        # if we take the [2, 3, 1] example, we have the following zips: (2, 3), (3, 1) -> we get [3 elems of 2 random values in each] + [1 elem with 3 random values in it] =>
        #                                                                                           [np.array([1.2, 2.3], np.array([4.5, 4.5]), np.array([5.6, 7.8]), np.array([3.3, 7.7, 9.9])]

    def print_everything(self):
        print(f"number of layers: {self.num_layers}")
        print(f"sizes: {self.sizes}")
        print(f"biases: {self.biases}")
        print(f"weights: {self.weights}")

    def feedforward(self, a):
        """
        calculates the output, given a as input, essentially turning the network on
        :param a: a numpy array of shape: (n, 1), for handwriting detection, would be a single column array with the pixel values
        :return: returns a numpy array with sizes[-1] values
        """
        # print(f"The shape of the input a is: {a.shape}, should of the format (n, 1)")
        a = np.reshape(a, (self.sizes[0], 1))
        # print(f"The shape of the input a is: {a.shape}, should of the format (n, 1)")
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a) + b)
            # print(f"a: {a}")
        return a

    def stochastic_gd(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Training the neural network using the stochastic gradient descent. Stochastic => batches of data.

        Working:
            for each epoch, it randomly shuffles the data, partitions it into the appropriate size of batches, and for each mini_batch we apply 1 step of gradient descent
            the single step of gradient descent is done by the step "self.update_mini_batch(mini_batch, eta)" -> updating the network's weights and biases
        :param training_data: list of tuples (x, y) representing training inputs and desired outputs
        :param epochs: the number of epochs to train for
        :param mini_batch_size: the size of the mini_batches to use for training
        :param eta: the learning rate
        :param test_data: if provided, then the network will be evaluated against the test data after each epoch, and partial progress will be printed out, this is slow
        :return: None, updates the neural network's weights and biases
        """
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k: k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)}/{n_test}")
            else:
                print(f"Epoch {j} completed")

    def update_mini_batch(self, mini_batch, eta):
        """
        Update the networks's weights and biases by applying gradient descent using backprop, to a single mini_batch.
        :param mini_batch: list of tuples, (x, y)
        :param eta: learning rate eta
        :return:
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]

    def sigmoid(self, z):
        return 1.0/ (1.0 + np.exp(-z))

    def sigmoid_prime(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def backprop(self, x, y):
        """

        :param x: input to the neural network, for handwriting NN example, array of pixel activation values
        :param y: the labelled output of what x should be, for handwriting NN example, a single digit [0-9]
        :return: a tuple (nabla_b, nabla_w) representing the gradient of the cost_function (gradient of cost_function calculates in which direction it increases the most)
                nabla_b and nabla_w are lists for each layer, just like self.weights and self.biases
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # feedforward to get the prediction of the NN for the given x input
        x = np.reshape(x, (self.sizes[0], 1))
        activation = x  # currently stores the input as the activation of layer 0
        activations = [x] # list to store all the activations, layer by layer, already consists of activation of layer 0, the input
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)

        # backward pass
        delta = self.cost_derivative(activations[-1], y) * self.sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())        # this is completely relying on the fact that the orders work, fix this

        for l in range(2, self.num_layers):
            # l ranges from 2 all the way till number of layers - 1, so if total number of layers = 3, so, l = 2... that's it
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_b, nabla_w

    def cost_derivative(self, activation, y):
        return activation - y

In [ ]:
n1 = Network([784, 100, 16, 10])
# n1.print_everything()

In [ ]:
# Loading the data
import pickle
import gzip

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    training_data, validation_data, test_data = u.load()
    f.close()
    return training_data, validation_data, test_data

data = load_data()

training_data = data[0]
testing_data = data[1]

training_data_inputs = training_data[0]
training_data_outputs = training_data[1]

prepped_training_data_inputs = np.array([np.reshape(elem, (784, 1)) for elem in training_data_inputs])

prepped_training_data_outputs = np.zeros((50000, 10, 1))  # Create a zero array
prepped_training_data_outputs[np.arange(50000), training_data_outputs, 0] = 1  # Set the correct index to 1

prepped_training_data = [(a, b) for a, b in zip(prepped_training_data_inputs, prepped_training_data_outputs)]

In [ ]:
n1.stochastic_gd(prepped_training_data, 200, 10, 0.01)

In [ ]:
def softmax(x):
    x_exp = np.exp(x - np.max(x))  # Stabilized exponentiation
    return x_exp / np.sum(x_exp)

In [ ]:
def get_number(prediction):
    return np.argmax(prediction)

In [ ]:
prepped_testing_data_inputs = np.array([np.reshape(elem, (784, 1)) for elem in testing_data_inputs])

In [ ]:
prepped_testing_data_outputs = testing_data[1]

In [ ]:
tot = 0
matches = 0
for i, testing_input in enumerate(prepped_testing_data_inputs):
    # print(testing_input)
    temp = n1.feedforward(testing_input)
    temp_soft = softmax(temp)
    predicted_number = get_number(temp_soft)
    tot += 1
    print(f"predicted_number: {predicted_number}, prepped_testing_data_outputs: {prepped_testing_data_outputs[i]}")
    if predicted_number != prepped_testing_data_outputs[i]:
        print("Did not match")
    else:
        matches += 1
        print("Matched")

print(matches/tot)